# Treatment Data Preprocessing
---

Reading and preprocessing treatment data of the eICU dataset from MIT with the data from over 139k patients collected in the US.

This notebook addresses the preprocessing of the following eICU tables:
* admissionDrug
* infusionDrug
* medication
* treatment
* intakeOutput

## Importing the necessary packages

In [1]:
import os                                  # os handles directory/workspace changes
import numpy as np                         # NumPy to handle numeric and NaN operations
import yaml                                # Save and load YAML files

In [2]:
# Debugging packages
import pixiedust                           # Debugging in Jupyter Notebook cells

Pixiedust database opened successfully


In [3]:
# Change to parent directory (presumably "Documents")
os.chdir("../../../..")

# Path to the CSV dataset files
data_path = 'Datasets/Thesis/eICU/uncompressed/'

# Path to the code files
project_path = 'GitHub/eICU-mortality-prediction/'

In [4]:
import modin.pandas as pd                  # Optimized distributed version of Pandas
import data_utils as du                    # Data science and machine learning relevant methods

Set the random seed for reproducibility

In [5]:
du.set_random_seed(42)

## Initialize variables

In [6]:
cat_feat = []                              # List of categorical features
cat_embed_feat = []                        # List of categorical features that will be embedded
cat_embed_feat_enum = dict()               # Dictionary of the enumerations of the categorical features that will be embedded

## Infusion drug data

### Read the data

In [7]:
infdrug_df = pd.read_csv(f'{data_path}original/infusionDrug.csv')
infdrug_df.head()

(pid=98873) DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
(pid=98867) DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.


,infusiondrugid,patientunitstayid,infusionoffset,drugname,drugrate,infusionrate,drugamount,volumeoffluid,patientweight
0,1953469,242040,457,Milrinone (mcg/kg/min),.43,11.8,20.0,100.0,91.7
1,1998443,242082,425,Norepinephrine (mcg/min),10.93,41.0,4.0,250.0,NaN
2,1968206,242082,125,Norepinephrine (mcg/min),7,26.3,4.0,250.0,NaN
3,1991487,242082,665,NS (ml/hr),200,200.0,NaN,NaN,NaN
4,1969910,242082,55,Norepinephrine (mcg/min),2.13,8.0,4.0,250.0,NaN


In [8]:
len(infdrug_df)

4803719

In [9]:
infdrug_df.patientunitstayid.nunique()

73547

Get an overview of the dataframe through the `describe` method:

In [10]:
infdrug_df.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
infusiondrugid,4803719.0,3.598207e+07,2.271653e+07,1950338.00,14920408.0,37517978.0,59148287.5,75335530.0
patientunitstayid,4803719.0,1.668055e+06,9.745644e+05,242040.00,839101.0,1471584.0,2757564.0,3353263.0
infusionoffset,4803719.0,6.223110e+03,9.051773e+03,-7362289.00,1176.0,3271.0,8007.0,205119.0
infusionrate,2101504.0,6.270923e+01,1.702578e+03,0.00,2.4,9.0,21.0,648900.0
drugamount,1694378.0,1.732015e+03,6.592233e+03,0.01,8.0,100.0,250.0,750000.0
volumeoffluid,1695077.0,1.753158e+02,1.201521e+02,0.00,100.0,100.0,250.0,5000.0
patientweight,443553.0,8.528778e+01,2.979588e+01,0.00,67.8,81.0,98.6,1113.0


In [11]:
infdrug_df.columns

Index(['infusiondrugid', 'patientunitstayid', 'infusionoffset', 'drugname',
       'drugrate', 'infusionrate', 'drugamount', 'volumeoffluid',
       'patientweight'],
      dtype='object')

In [12]:
infdrug_df.dtypes

infusiondrugid         int64
patientunitstayid      int64
infusionoffset         int64
drugname              object
drugrate              object
infusionrate         float64
drugamount           float64
volumeoffluid        float64
patientweight        float64
dtype: object

### Check for missing values

In [13]:
du.search_explore.dataframe_missing_values(infdrug_df)

,column_name,percent_missing
infusiondrugid,infusiondrugid,0.000000
patientunitstayid,patientunitstayid,0.000000
infusionoffset,infusionoffset,0.000000
drugname,drugname,0.000021
drugrate,drugrate,0.323041
infusionrate,infusionrate,56.252562
volumeoffluid,volumeoffluid,64.713236
drugamount,drugamount,64.727787
patientweight,patientweight,90.766467


### Remove unneeded features

Besides removing the row ID `infusiondrugid`, I'm also removing `infusionrate`, `volumeoffluid` and `drugamount` as they seem redundant with `drugrate` although with a lot more missing values.

In [14]:
infdrug_df = infdrug_df.drop(['infusiondrugid', 'infusionrate', 'volumeoffluid',
                              'drugamount', 'patientweight'], axis=1)
infdrug_df.head()

,patientunitstayid,infusionoffset,drugname,drugrate
0,242040,457,Milrinone (mcg/kg/min),.43
1,242082,425,Norepinephrine (mcg/min),10.93
2,242082,125,Norepinephrine (mcg/min),7
3,242082,665,NS (ml/hr),200
4,242082,55,Norepinephrine (mcg/min),2.13


### Remove string drug rate values

In [15]:
infdrug_df[infdrug_df.drugrate.map(du.utils.is_definitely_string)].head()

,patientunitstayid,infusionoffset,drugname,drugrate
460,245984,1348,Dopamine (ml/hr),UD
905,248581,59632,Heparin (Unknown),ERROR
923,248581,61007,Sodium Chloride 0.9% 500 ml (ml/hr),UD
931,248581,54083,Heparin (Unknown),ERROR
937,248581,60280,Heparin (Unknown),ERROR


In [16]:
infdrug_df[infdrug_df.drugrate.map(du.utils.is_definitely_string)].drugrate.value_counts()

To request implementation, send an email to feature_requests@modin.org.


ERROR                         2831
OFF                             24
UD                              13
off                             11
Nafcillin 14.5ml/hr             10
                              ... 
running into venous sheath       1
Angiomax 1.75 mg/kg/h            1
Integrillin 2 mcg/kg/min         1
pantoprazole off                 1
off\.br\                         1
Length: 64, dtype: int64

In [17]:
infdrug_df.drugrate = infdrug_df.drugrate.map(lambda x: np.nan if du.utils.is_definitely_string(x) else x)
infdrug_df.head()

,patientunitstayid,infusionoffset,drugname,drugrate
0,242040,457,Milrinone (mcg/kg/min),.43
1,242082,425,Norepinephrine (mcg/min),10.93
2,242082,125,Norepinephrine (mcg/min),7
3,242082,665,NS (ml/hr),200
4,242082,55,Norepinephrine (mcg/min),2.13


In [18]:
infdrug_df.patientunitstayid = infdrug_df.patientunitstayid.astype(int)
infdrug_df.infusionoffset = infdrug_df.infusionoffset.astype(int)
infdrug_df.drugname = infdrug_df.drugname.astype(str)
infdrug_df.drugrate = infdrug_df.drugrate.astype(float)
infdrug_df.head()

,patientunitstayid,infusionoffset,drugname,drugrate
0,242040,457,Milrinone (mcg/kg/min),.43
1,242082,425,Norepinephrine (mcg/min),10.93
2,242082,125,Norepinephrine (mcg/min),7
3,242082,665,NS (ml/hr),200
4,242082,55,Norepinephrine (mcg/min),2.13


### Discretize categorical features

Convert binary categorical features into simple numberings, one hot encode features with a low number of categories (in this case, 5) and enumerate sparse categorical features that will be embedded.

#### Separate and prepare features for embedding

Identify categorical features that have more than 5 unique categories, which will go through an embedding layer afterwards, and enumerate them.

Update list of categorical features and add those that will need embedding (features with more than 5 unique values):

In [19]:
new_cat_feat = ['drugname']
[cat_feat.append(col) for col in new_cat_feat]

[None]

In [20]:
cat_feat_nunique = [infdrug_df[feature].nunique() for feature in new_cat_feat]
cat_feat_nunique

[3961]

In [21]:
new_cat_embed_feat = []
for i in range(len(new_cat_feat)):
    if cat_feat_nunique[i] > 5:
        # Add feature to the list of those that will be embedded
        cat_embed_feat.append(new_cat_feat[i])
        new_cat_embed_feat.append(new_cat_feat[i])

In [22]:
infdrug_df[new_cat_feat].head()

,drugname
0,Milrinone (mcg/kg/min)
1,Norepinephrine (mcg/min)
2,Norepinephrine (mcg/min)
3,NS (ml/hr)
4,Norepinephrine (mcg/min)


In [23]:
for i in range(len(new_cat_embed_feat)):
    feature = new_cat_embed_feat[i]
    # Prepare for embedding, i.e. enumerate categories
    infdrug_df[feature], cat_embed_feat_enum[feature] = du.embedding.enum_categorical_feature(infdrug_df, feature, nan_value=0)

In [24]:
infdrug_df[new_cat_feat].head()

,drugname
0,1
1,2
2,2
3,3
4,2


In [25]:
cat_embed_feat_enum

{'drugname': {'milrinone_(mcg/kg/min)': 1,
  'norepinephrine_(mcg/min)': 2,
  'ns_(ml/hr)': 3,
  'propofol_(mcg/kg/min)': 4,
  'lidocaine_(mg/min)': 5,
  'heparin_(units/hr)': 6,
  'nitroglycerin_(mcg/min)': 7,
  'diltiazem_(mg/hr)': 8,
  'dopamine_(mcg/kg/min)': 9,
  'amiodarone_(mg/min)': 10,
  'eptifibatide_(mcg/kg/min)': 11,
  'propofol_(ml/hr)': 12,
  'banana_bag_(ml/hr)': 13,
  'ns/20kcl_(ml/hr)': 14,
  'propofol_(mcg/min)': 15,
  'vasopressin_(units/min)': 16,
  'epinephrine_(mcg/min)': 17,
  'dexmedetomidine_inj_400_mcg_in_dextrose_5%_100_ml_(mcg/kg/hr)': 18,
  'dexmedetomidine_(mcg/kg/hr)': 19,
  'fentanyl_(mcg/hr)': 20,
  'fentanyl_(ml/hr)': 21,
  'fentanyl_(mcg/kg/min)': 22,
  'nicardipine_(mg/hr)': 23,
  'piperacillin-tazo_3.375gm_50_ml_(ml/hr)': 24,
  'ceftriaxone_2_gm/d5w_250_ml_(ml/hr)': 25,
  '0.9_ns_(ml/hr)': 26,
  'insulin_(units/hr)': 27,
  'esmolol_(mcg/kg/min)': 28,
  'phenylephrine_(mcg/min)': 29,
  'd5w/3_amps_hco3_(ml/hr)': 30,
  'insulin_(ml/hr)': 31,
  'd51/2_

In [26]:
infdrug_df[new_cat_feat].dtypes

drugname    int64
dtype: object

#### Save enumeration encoding mapping

Save the dictionary that maps from the original categories/strings to the new numerical encondings.

In [27]:
stream = open(f'{data_path}/cleaned/cat_embed_feat_enum_inf_drug.yaml', 'w')
yaml.dump(cat_embed_feat_enum, stream, default_flow_style=False)

### Create the timestamp feature and sort

Create the timestamp (`ts`) feature:

In [28]:
infdrug_df = infdrug_df.rename(columns={'infusionoffset': 'ts'})
infdrug_df.head()

,patientunitstayid,ts,drugname,drugrate
0,242040,457,1,.43
1,242082,425,2,10.93
2,242082,125,2,7
3,242082,665,3,200
4,242082,55,2,2.13


Remove duplicate rows:

In [29]:
len(infdrug_df)

4803719

In [30]:
infdrug_df = infdrug_df.drop_duplicates()
infdrug_df.head()

,patientunitstayid,ts,drugname,drugrate
0,242040,457,1,.43
1,242082,425,2,10.93
2,242082,125,2,7
3,242082,665,3,200
4,242082,55,2,2.13


In [31]:
len(infdrug_df)

4797983

Sort by `ts` so as to be easier to merge with other dataframes later:

In [32]:
infdrug_df = infdrug_df.sort_values('ts')
infdrug_df.head(6)

,patientunitstayid,ts,drugname,drugrate
95016,443290,-7362289,8,5
4707737,3215842,-525445,23,5
1511900,964293,-381117,20,25
1577798,975371,-86490,4,10
4719152,3222463,-67922,7,60
4719153,3222463,-67352,7,90


Convert dataframe to Pandas, as the next cells aren't working properly with Modin:

In [33]:
infdrug_df.dtypes

patientunitstayid     int64
ts                    int64
drugname              int64
drugrate             object
dtype: object

In [34]:
infdrug_df, pd = du.utils.convert_dataframe(infdrug_df, to='pandas')

In [35]:
type(infdrug_df)

pandas.core.frame.DataFrame

In [36]:
infdrug_df.dtypes

patientunitstayid    object
ts                   object
drugname             object
drugrate             object
dtype: object

Check for possible multiple rows with the same unit stay ID and timestamp:

In [37]:
infdrug_df.groupby(['patientunitstayid', 'ts']).count().nlargest(columns='drugname', n=5).head()

drugname  drugrate
patientunitstayid ts                       
1785711           14463        17        17
                  8995         15        15
                  14021        15        15
                  7424         14        14
                  7987         14        14

In [38]:
infdrug_df[infdrug_df.patientunitstayid == 1785711].head(20)

,patientunitstayid,ts,drugname,drugrate
39195,1785711,-20,27,10
39207,1785711,-20,122,0.17
39216,1785711,-20,19,0.17
39221,1785711,-20,10,1
39287,1785711,-20,2,3.01
39553,1785711,-20,69,4.1
482874,1785711,407,27,5
482875,1785711,407,2,2.45
482876,1785711,407,10,1
482881,1785711,407,69,4.92


We can see that there are up to 17 categories per set of `patientunitstayid` and `ts`. As such, we must join them. But first, as we shouldn't mix absolute values of drug rates from different drugs, we better normalize it first.

### Normalize data

In [39]:
infdrug_df.drugrate = infdrug_df.drugrate.astype(float)

In [40]:
infdrug_df_norm = du.data_processing.normalize_data(infdrug_df, columns_to_normalize=False,
                                                    columns_to_normalize_categ=[('drugname', 'drugrate')],
                                                    inplace=True)
infdrug_df_norm.head()

z-score normalizing columns [('drugname', 'drugrate')] by their associated categories...


,patientunitstayid,ts,drugname,drugrate
0,443290,-7362289,8,-0.398766
1,3215842,-525445,23,0.048985
2,964293,-381117,20,-0.016957
3,975371,-86490,4,-0.813303
4,3222463,-67922,7,0.780472


Prevent infinite drug rate values:

In [41]:
infdrug_df_norm = infdrug_df_norm.replace(to_replace=np.inf, value=0)

In [42]:
infdrug_df_norm.drugrate.max()

534.895379210853

### Join rows that have the same IDs

In [ ]:
infdrug_df_norm = du.embedding.join_categorical_enum(infdrug_df_norm, new_cat_embed_feat, inplace=True)
infdrug_df_norm.head()

Concatenating categorical encodings...


Reconvert dataframe to Modin:

In [ ]:
infdrug_df_norm, pd = du.utils.convert_dataframe(infdrug_df_norm, to='modin')

In [ ]:
type(infdrug_df_norm)

In [ ]:
infdrug_df_norm.dtypes

In [ ]:
infdrug_df_norm.groupby(['patientunitstayid', 'ts']).count().nlargest(columns='drugname', n=5).head()

In [ ]:
infdrug_df_norm[infdrug_df_norm.patientunitstayid == 1785711].head(20)

Comparing the output from the two previous cells with what we had before the `join_categorical_enum` method, we can see that all rows with duplicate IDs have been successfully joined.

### Rename columns

In [ ]:
infdrug_df = infdrug_df.rename(columns={'drugname': 'infusion_drugname',
                                        'drugrate': 'infusion_drugrate'})
infdrug_df.head()

In [ ]:
infdrug_df_norm = infdrug_df_norm.rename(columns={'drugname': 'infusion_drugname',
                                                  'drugrate': 'infusion_drugrate'})
infdrug_df_norm.head()

### Clean column names

Standardize all column names to be on lower case, have spaces replaced by underscores and remove comas.

In [ ]:
infdrug_df.columns = du.data_processing.clean_naming(infdrug_df.columns)
infdrug_df_norm.columns = du.data_processing.clean_naming(infdrug_df_norm.columns)
infdrug_df_norm.head()

### Save the dataframe

Save the dataframe before normalizing:

In [ ]:
infdrug_df.to_csv(f'{data_path}cleaned/unnormalized/infusionDrug.csv')

Save the dataframe after normalizing:

In [ ]:
infdrug_df_norm.to_csv(f'{data_path}cleaned/normalized/infusionDrug.csv')

Confirm that everything is ok through the `describe` method:

In [ ]:
infdrug_df_norm.describe().transpose()

## Admission drug data

In [ ]:
infdrug_df_norm[infdrug_df_norm.infusion_drugrate == np.inf].head()

In [ ]:
infdrug_df_norm.replace(to_replace=np.inf, value=0).infusion_drugrate.max()

### Read the data

In [ ]:
admsdrug_df = pd.read_csv(f'{data_path}original/admissionDrug.csv')
admsdrug_df.head()

In [ ]:
len(admsdrug_df)

In [ ]:
admsdrug_df.patientunitstayid.nunique()

There's not much admission drug data (only around 20% of the unit stays have this data). However, it might be useful, considering also that it complements the medication table.

Get an overview of the dataframe through the `describe` method:

In [ ]:
admsdrug_df.describe().transpose()

In [ ]:
admsdrug_df.columns

In [ ]:
admsdrug_df.dtypes

### Check for missing values

In [ ]:
du.search_explore.dataframe_missing_values(admsdrug_df)

### Remove unneeded features

In [ ]:
admsdrug_df.drugname.value_counts()

In [ ]:
admsdrug_df.drughiclseqno.value_counts()

In [ ]:
admsdrug_df.drugnotetype.value_counts()

In [ ]:
admsdrug_df.drugdosage.value_counts()

In [ ]:
admsdrug_df.drugunit.value_counts()

In [ ]:
admsdrug_df.drugadmitfrequency.value_counts()

In [ ]:
admsdrug_df[admsdrug_df.drugdosage == 0].head(20)

In [ ]:
admsdrug_df[admsdrug_df.drugdosage == 0].drugunit.value_counts()

In [ ]:
admsdrug_df[admsdrug_df.drugdosage == 0].drugadmitfrequency.value_counts()

In [ ]:
admsdrug_df[admsdrug_df.drugunit == ' '].drugdosage.value_counts()

Oddly, `drugunit` and `drugadmitfrequency` have several blank values. At the same time, when this happens, `drugdosage` tends to be 0 (which is also an unrealistic value). Considering that no NaNs are reported, these blanks and zeros probably represent missing values.

Besides removing irrelevant or hospital staff related data (e.g. `usertype`), I'm also removing the `drugname` column, which is redundant with the codes `drughiclseqno`, while also being brand dependant.

In [ ]:
admsdrug_df = admsdrug_df[['patientunitstayid', 'drugoffset', 'drugdosage',
                           'drugunit', 'drugadmitfrequency', 'drughiclseqno']]
admsdrug_df.head()

### Fix missing values representation

Replace blank and unrealistic zero values with NaNs.

In [ ]:
admsdrug_df.drugdosage = admsdrug_df.drugdosage.replace(to_replace=0, value=np.nan)
admsdrug_df.drugunit = admsdrug_df.drugunit.replace(to_replace=' ', value=np.nan)
admsdrug_df.drugadmitfrequency = admsdrug_df.drugadmitfrequency.replace(to_replace=' ', value=np.nan)
admsdrug_df.head()

In [ ]:
du.search_explore.dataframe_missing_values(admsdrug_df)

### Discretize categorical features

Convert binary categorical features into simple numberings, one hot encode features with a low number of categories (in this case, 5) and enumerate sparse categorical features that will be embedded.

#### Separate and prepare features for embedding

Identify categorical features that have more than 5 unique categories, which will go through an embedding layer afterwards, and enumerate them.

In the case of microbiology data, we're also going to embed the antibiotic `sensitivitylevel`, not because it has many categories, but because there can be several rows of data per timestamp (which would be impractical on one hot encoded data).

Update list of categorical features and add those that will need embedding (features with more than 5 unique values):

In [ ]:
new_cat_feat = ['drugunit', 'drugadmitfrequency', 'drughiclseqno']
[cat_feat.append(col) for col in new_cat_feat]

In [ ]:
cat_feat_nunique = [admsdrug_df[feature].nunique() for feature in new_cat_feat]
cat_feat_nunique

In [ ]:
new_cat_embed_feat = []
for i in range(len(new_cat_feat)):
    if cat_feat_nunique[i] > 5:
        # Add feature to the list of those that will be embedded
        cat_embed_feat.append(new_cat_feat[i])
        new_cat_embed_feat.append(new_cat_feat[i])

In [ ]:
admsdrug_df[new_cat_feat].head()

In [ ]:
for i in range(len(new_cat_embed_feat)):
    feature = new_cat_embed_feat[i]
    # Skip the 'drughiclseqno' from enumeration encoding
    if feature == 'drughiclseqno':
        continue
    # Prepare for embedding, i.e. enumerate categories
    admsdrug_df[feature], cat_embed_feat_enum[feature] = du.embedding.enum_categorical_feature(admsdrug_df, feature, nan_value=0)

In [ ]:
admsdrug_df[new_cat_feat].head()

In [ ]:
cat_embed_feat_enum

In [ ]:
admsdrug_df[new_cat_feat].dtypes

#### Save enumeration encoding mapping

Save the dictionary that maps from the original categories/strings to the new numerical encondings.

In [ ]:
stream = open(f'{data_path}/cleaned/cat_embed_feat_enum_adms_drug.yaml', 'w')
yaml.dump(cat_embed_feat_enum, stream, default_flow_style=False)

### Create the timestamp feature and sort

Convert dataframe to Pandas, as the next cells aren't working properly with Modin:

In [ ]:
admsdrug_df, pd = du.utils.convert_dataframe(admsdrug_df, to='pandas')

In [ ]:
type(admsdrug_df)

Create the timestamp (`ts`) feature:

In [ ]:
admsdrug_df = admsdrug_df.rename(columns={'drugoffset': 'ts'})
admsdrug_df.head()

Remove duplicate rows:

In [ ]:
len(admsdrug_df)

In [ ]:
admsdrug_df = admsdrug_df.drop_duplicates()
admsdrug_df.head()

In [ ]:
len(admsdrug_df)

Sort by `ts` so as to be easier to merge with other dataframes later:

In [ ]:
admsdrug_df = admsdrug_df.sort_values('ts')
admsdrug_df.head()

Check for possible multiple rows with the same unit stay ID and timestamp:

In [ ]:
admsdrug_df.groupby(['patientunitstayid', 'ts']).count().nlargest(columns='drughiclseqno', n=5).head()

In [ ]:
admsdrug_df[admsdrug_df.patientunitstayid == 2346930].head(10)

We can see that there are up to 48 categories per set of `patientunitstayid` and `ts`. As such, we must join them. But first, we need to normalize the dosage by the respective sets of drug code and units, so as to avoid mixing different absolute values.

### Normalize data

In [ ]:
admsdrug_df_norm = du.data_processing.normalize_data(admsdrug_df_norm, columns_to_normalize=False,
                                                     columns_to_normalize_cat=[(['drughiclseqno', 'drugunit'], 'drugdosage')],
                                                     inplace=True)
admsdrug_df_norm.head()

In [ ]:
admsdrug_df_norm = admsdrug_df_norm.sort_values('ts')
admsdrug_df_norm.head()

### Join rows that have the same IDs

Even after removing duplicates rows, there are still some that have different information for the same ID and timestamp. We have to concatenate the categorical enumerations.

Convert dataframe to Pandas, as the groupby operation in `join_categorical_enum` isn't working properly with Modin:

In [ ]:
admsdrug_df_norm, pd = du.utils.convert_dataframe(admsdrug_df_norm, to='pandas')

In [ ]:
type(admsdrug_df_norm)

In [ ]:
admsdrug_df_norm = du.embedding.join_categorical_enum(admsdrug_df_norm, new_cat_embed_feat, inplace=True)
admsdrug_df_norm.head()

Reconvert dataframe to Modin:

In [ ]:
admsdrug_df_norm, pd = du.utils.convert_dataframe(admsdrug_df_norm, to='modin')

In [ ]:
type(admsdrug_df_norm)

In [ ]:
admsdrug_df_norm.dtypes

In [ ]:
admsdrug_df_norm.groupby(['patientunitstayid', 'ts']).count().nlargest(columns='drughiclseqno', n=5).head()

In [ ]:
admsdrug_df_norm[admsdrug_df_norm.patientunitstayid == 2346930].head(10)

Comparing the output from the two previous cells with what we had before the `join_categorical_enum` method, we can see that all rows with duplicate IDs have been successfully joined.

### Clean column names

Standardize all column names to be on lower case, have spaces replaced by underscores and remove comas.

In [ ]:
admsdrug_df.columns = du.data_processing.clean_naming(admsdrug_df.columns)
admsdrug_df_norm.columns = du.data_processing.clean_naming(admsdrug_df_norm.columns)
admsdrug_df_norm.head()

### Save the dataframe

Save the dataframe before normalizing:

In [ ]:
admsdrug_df.to_csv(f'{data_path}cleaned/unnormalized/admissionDrug.csv')

Save the dataframe after normalizing:

In [ ]:
admsdrug_df_norm.to_csv(f'{data_path}cleaned/normalized/admissionDrug.csv')

Confirm that everything is ok through the `describe` method:

In [ ]:
admsdrug_df_norm.describe().transpose()

## Medication data

### Read the data

In [ ]:
med_df = pd.read_csv(f'{data_path}original/medication.csv', dtype={'loadingdose': 'object'})
med_df.head()

In [ ]:
len(med_df)

In [ ]:
med_df.patientunitstayid.nunique()

There's not much admission drug data (only around 20% of the unit stays have this data). However, it might be useful, considering also that it complements the medication table.

Get an overview of the dataframe through the `describe` method:

In [ ]:
med_df.describe().transpose()

In [ ]:
med_df.columns

In [ ]:
med_df.dtypes

### Check for missing values

In [ ]:
du.search_explore.dataframe_missing_values(med_df)

### Remove unneeded features

In [ ]:
med_df.drugname.value_counts()

In [ ]:
med_df.drughiclseqno.value_counts()

In [ ]:
med_df.dosage.value_counts()

In [ ]:
med_df.frequency.value_counts()

In [ ]:
med_df.drugstartoffset.value_counts()

In [ ]:
med_df[med_df.drugstartoffset == 0].head()

Besides removing less interesting data (e.g. `drugivadmixture`), I'm also removing the `drugname` column, which is redundant with the codes `drughiclseqno`, while also being brand dependant.

In [ ]:
med_df = med_df[['patientunitstayid', 'drugstartoffset', 'drugstopoffset',
                 'drugordercancelled', 'dosage', 'frequency', 'drughiclseqno']]
med_df.head()

### Remove rows of which the drug has been cancelled or not specified

In [ ]:
med_df.drugordercancelled.value_counts()

In [ ]:
med_df = med_df[~((med_df.drugordercancelled == 'Yes') | (np.isnan(med_df.drughiclseqno)))]
med_df.head()

Remove the now unneeded `drugordercancelled` column:

In [ ]:
med_df = med_df.drop('drugordercancelled', axis=1)
med_df.head()

In [ ]:
du.search_explore.dataframe_missing_values(med_df)

### Separating units from dosage

In order to properly take into account the dosage quantities, as well as to standardize according to other tables like admission drugs, we should take the original `dosage` column and separate it to just the `drugdosage` values and the `drugunit`.

No need to create a separate `pyxis` feature, which would indicate the use of the popular automated medications manager, as the frequency embedding will have that into account.

Create dosage and unit features:

In [ ]:
med_df['drugdosage'] = np.nan
med_df['drugunit'] = np.nan
med_df.head()

Get the dosage and unit values for each row:

In [ ]:
med_df[['drugdosage', 'drugunit']] = med_df.apply(du.data_processing.set_dosage_and_units, axis=1, result_type='expand')
med_df.head()

Remove the now unneeded `dosage` column:

In [ ]:
med_df = med_df.drop('dosage', axis=1)
med_df.head()

### Discretize categorical features

Convert binary categorical features into simple numberings, one hot encode features with a low number of categories (in this case, 5) and enumerate sparse categorical features that will be embedded.

#### Separate and prepare features for embedding

Identify categorical features that have more than 5 unique categories, which will go through an embedding layer afterwards, and enumerate them.

In the case of microbiology data, we're also going to embed the antibiotic `sensitivitylevel`, not because it has many categories, but because there can be several rows of data per timestamp (which would be impractical on one hot encoded data).

Update list of categorical features and add those that will need embedding (features with more than 5 unique values):

In [ ]:
new_cat_feat = ['drugunit', 'frequency', 'drughiclseqno']
[cat_feat.append(col) for col in new_cat_feat]

In [ ]:
cat_feat_nunique = [med_df[feature].nunique() for feature in new_cat_feat]
cat_feat_nunique

In [ ]:
new_cat_embed_feat = []
for i in range(len(new_cat_feat)):
    if cat_feat_nunique[i] > 5:
        # Add feature to the list of those that will be embedded
        cat_embed_feat.append(new_cat_feat[i])
        new_cat_embed_feat.append(new_cat_feat[i])

In [ ]:
med_df[new_cat_feat].head()

In [ ]:
for i in range(len(new_cat_embed_feat)):
    feature = new_cat_embed_feat[i]
    # Skip the 'drughiclseqno' from enumeration encoding
    if feature == 'drughiclseqno':
        continue
    # Prepare for embedding, i.e. enumerate categories
    med_df[feature], cat_embed_feat_enum[feature] = du.embedding.enum_categorical_feature(med_df, feature, nan_value=0)

In [ ]:
med_df[new_cat_feat].head()

In [ ]:
cat_embed_feat_enum

In [ ]:
med_df[new_cat_feat].dtypes

#### Save enumeration encoding mapping

Save the dictionary that maps from the original categories/strings to the new numerical encondings.

In [ ]:
stream = open(f'{data_path}/cleaned/cat_embed_feat_enum_med.yaml', 'w')
yaml.dump(cat_embed_feat_enum, stream, default_flow_style=False)

### Create drug stop event

Add a timestamp corresponding to when each patient stops taking each medication.

Duplicate every row, so as to create a discharge event:

In [ ]:
new_df = med_df.copy()
new_df.head()

Set the new dataframe's rows to have the drug stop timestamp, with no more information on the drug that was being used:

In [ ]:
new_df.drugstartoffset = new_df.drugstopoffset
new_df.drugunit = np.nan
new_df.drugdosage = np.nan
new_df.frequency = np.nan
new_df.drughiclseqno = np.nan
new_df.head()

Join the new rows to the remaining dataframe:

In [ ]:
med_df = med_df.append(new_df)
med_df.head()

Remove the now unneeded medication stop column:

In [ ]:
med_df = med_df.drop('drugstopoffset', axis=1)
med_df.head(6)

### Create the timestamp feature and sort

Create the timestamp (`ts`) feature:

In [ ]:
med_df = med_df.rename(columns={'drugstopoffset': 'ts'})
med_df.head()

Remove duplicate rows:

In [ ]:
len(med_df)

In [ ]:
med_df = med_df.drop_duplicates()
med_df.head()

In [ ]:
len(med_df)

Sort by `ts` so as to be easier to merge with other dataframes later:

In [ ]:
med_df = med_df.sort_values('ts')
med_df.head()

Check for possible multiple rows with the same unit stay ID and timestamp:

In [ ]:
med_df.groupby(['patientunitstayid', 'ts']).count().nlargest(columns='drughiclseqno', n=5).head()

In [ ]:
med_df[med_df.patientunitstayid == 979183].head(10)

We can see that there are up to 41 categories per set of `patientunitstayid` and `ts`. As such, we must join them. But first, we need to normalize the dosage by the respective sets of drug code and units, so as to avoid mixing different absolute values.

### Normalize data

In [ ]:
med_df_norm = du.data_processing.normalize_data(med_df_norm, columns_to_normalize=False,
                                                columns_to_normalize_cat=[(['drughiclseqno', 'drugunit'], 'drugdosage')],
                                                inplace=True)
med_df_norm.head()

In [ ]:
med_df_norm = med_df_norm.sort_values('ts')
med_df_norm.head()

### Join rows that have the same IDs

Even after removing duplicates rows, there are still some that have different information for the same ID and timestamp. We have to concatenate the categorical enumerations.

In [ ]:
list(set(med_df_norm.columns) - set(new_cat_embed_feat) - set(['patientunitstayid', 'ts']))

Convert dataframe to Pandas, as the groupby operation in `join_categorical_enum` isn't working properly with Modin:

In [ ]:
med_df_norm, pd = du.utils.convert_dataframe(med_df, to='pandas')

In [ ]:
type(med_df_norm)

In [ ]:
med_df_norm = du.embedding.join_categorical_enum(med_df_norm, new_cat_embed_feat, inplace=True)
med_df_norm.head()

Reconvert dataframe to Modin:

In [ ]:
med_df_norm, pd = du.utils.convert_dataframe(med_df, to='modin')

In [ ]:
type(med_df_norm)

In [ ]:
med_df_norm.dtypes

In [ ]:
med_df_norm.groupby(['patientunitstayid', 'ts']).count().nlargest(columns='drughiclseqno', n=5).head()

In [ ]:
med_df_norm[med_df_norm.patientunitstayid == 979183].head(10)

Comparing the output from the two previous cells with what we had before the `join_categorical_enum` method, we can see that all rows with duplicate IDs have been successfully joined.

### Rename columns

In [ ]:
med_df_norm = med_df_norm.rename(columns={'frequency':'drugadmitfrequency'})
med_df_norm.head()

### Clean column names

Standardize all column names to be on lower case, have spaces replaced by underscores and remove comas.

In [ ]:
med_df.columns = du.data_processing.clean_naming(med_df.columns)
med_df_norm.columns = du.data_processing.clean_naming(med_df_norm.columns)
med_df_norm.head()

### Save the dataframe

Save the dataframe before normalizing:

In [ ]:
med_df.to_csv(f'{data_path}cleaned/unnormalized/medication.csv')

Save the dataframe after normalizing:

In [ ]:
med_df_norm.to_csv(f'{data_path}cleaned/normalized/medication.csv')

Confirm that everything is ok through the `describe` method:

In [ ]:
med_df_norm.describe().transpose()

In [ ]:
med_df.nlargest(columns='drugdosage', n=5)

Although the `drugdosage` looks good on mean (close to 0) and standard deviation (close to 1), it has very large magnitude minimum (-88.9) and maximum (174.1) values. Furthermore, these don't seem to be because of NaN values, whose groupby normalization could have been unideal. As such, it's hard to say if these are outliers or realistic values.

[TODO] Check if these very large extreme dosage values make sense and, if not, try to fix them.

## Treatment data

### Read the data

In [ ]:
treat_df = pd.read_csv(f'{data_path}original/treatment.csv')
treat_df.head()

In [ ]:
len(treat_df)

In [ ]:
treat_df.patientunitstayid.nunique()

Get an overview of the dataframe through the `describe` method:

In [ ]:
treat_df.describe().transpose()

In [ ]:
treat_df.columns

In [ ]:
treat_df.dtypes

### Check for missing values

In [ ]:
du.search_explore.dataframe_missing_values(treat_df)

### Remove unneeded features

Besides the usual removal of row identifier, `treatmentid`, I'm also removing `activeupondischarge`, as we don't have complete information as to when diagnosis end.

In [ ]:
treat_df = treat_df.drop(['treatmentid', 'activeupondischarge'], axis=1)
treat_df.head()

### Separate high level diagnosis

In [ ]:
treat_df.treatmentstring.value_counts()

In [ ]:
treat_df.treatmentstring.map(lambda x: x.split('|')).head()

In [ ]:
treat_df.treatmentstring.map(lambda x: len(x.split('|'))).min()

In [ ]:
treat_df.treatmentstring.map(lambda x: len(x.split('|'))).max()

There are always at least 3 higher level diagnosis. It could be beneficial to extract those first 3 levels to separate features, with the last one getting values until the end of the string, so as to avoid the need for the model to learn similarities that are already known.

In [ ]:
treat_df.treatmentstring.apply(lambda x: du.search_explore.get_element_from_split(x, 0, separator='|')).value_counts()

In [ ]:
treat_df.treatmentstring.apply(lambda x: du.search_explore.get_element_from_split(x, 1, separator='|')).value_counts()

In [ ]:
treat_df.treatmentstring.apply(lambda x: du.search_explore.get_element_from_split(x, 2, separator='|')).value_counts()

In [ ]:
treat_df.treatmentstring.apply(lambda x: du.search_explore.get_element_from_split(x, 3, separator='|')).value_counts()

In [ ]:
treat_df.treatmentstring.apply(lambda x: du.search_explore.get_element_from_split(x, 4, separator='|')).value_counts()

In [ ]:
treat_df.treatmentstring.apply(lambda x: du.search_explore.get_element_from_split(x, 5, separator='|')).value_counts()

<!-- There are always 8 levels of the notes. As the first 6 ones are essentially always the same ("notes/Progress Notes/Social History / Family History/Social History/Social History/"), it's best to just preserve the 7th one and isolate the 8th in a new feature. This way, the split provides further insight to the model on similar notes. However, it's also worth taking note that the 8th level of `notepath` seems to be identical to the feature `notevalue`. We'll look more into it later. -->

In [ ]:
treat_df['treatmenttype'] = treat_df.treatmentstring.apply(lambda x: du.search_explore.get_element_from_split(x, 0, separator='|'))
treat_df['treatmenttherapy'] = treat_df.treatmentstring.apply(lambda x: du.search_explore.get_element_from_split(x, 1, separator='|'))
treat_df['treatmentdetails'] = treat_df.treatmentstring.apply(lambda x: du.search_explore.get_element_from_split(x, 2, separator='|', till_the_end=True))
treat_df.head()

Remove the now redundant `treatmentstring` column:

In [ ]:
treat_df = treat_df.drop('treatmentstring', axis=1)
treat_df.head()

In [ ]:
treat_df.treatmenttype.value_counts()

In [ ]:
treat_df.treatmenttherapy.value_counts()

In [ ]:
treat_df.treatmentdetails.value_counts()

### Discretize categorical features

Convert binary categorical features into simple numberings, one hot encode features with a low number of categories (in this case, 5) and enumerate sparse categorical features that will be embedded.

#### Separate and prepare features for embedding

Identify categorical features that have more than 5 unique categories, which will go through an embedding layer afterwards, and enumerate them.

Update list of categorical features and add those that will need embedding (features with more than 5 unique values):

In [ ]:
new_cat_feat = ['treatmenttype', 'treatmenttherapy', 'treatmentdetails']
[cat_feat.append(col) for col in new_cat_feat]

In [ ]:
cat_feat_nunique = [treat_df[feature].nunique() for feature in new_cat_feat]
cat_feat_nunique

In [ ]:
new_cat_embed_feat = []
for i in range(len(new_cat_feat)):
    if cat_feat_nunique[i] > 5:
        # Add feature to the list of those that will be embedded
        cat_embed_feat.append(new_cat_feat[i])
        new_cat_embed_feat.append(new_cat_feat[i])

In [ ]:
treat_df[new_cat_feat].head()

In [ ]:
for i in range(len(new_cat_embed_feat)):
    feature = new_cat_embed_feat[i]
    # Prepare for embedding, i.e. enumerate categories
    treat_df[feature], cat_embed_feat_enum[feature] = du.embedding.enum_categorical_feature(treat_df, feature, nan_value=0)

In [ ]:
treat_df[new_cat_feat].head()

In [ ]:
cat_embed_feat_enum

In [ ]:
treat_df[new_cat_feat].dtypes

#### Save enumeration encoding mapping

Save the dictionary that maps from the original categories/strings to the new numerical encondings.

In [ ]:
stream = open(f'{data_path}/cleaned/cat_embed_feat_enum_treat.yaml', 'w')
yaml.dump(cat_embed_feat_enum, stream, default_flow_style=False)

### Create the timestamp feature and sort

Create the timestamp (`ts`) feature:

In [ ]:
treat_df = treat_df.rename(columns={'treatmentoffset': 'ts'})
treat_df.head()

Remove duplicate rows:

In [ ]:
len(treat_df)

In [ ]:
treat_df = treat_df.drop_duplicates()
treat_df.head()

In [ ]:
len(treat_df)

Sort by `ts` so as to be easier to merge with other dataframes later:

In [ ]:
treat_df = treat_df.sort_values('ts')
treat_df.head()

Check for possible multiple rows with the same unit stay ID and timestamp:

In [ ]:
treat_df.groupby(['patientunitstayid', 'ts']).count().nlargest(columns='treatmenttype', n=5).head()

In [ ]:
treat_df[treat_df.patientunitstayid == 1352520].head(10)

We can see that there are up to 105 categories per set of `patientunitstayid` and `ts`. As such, we must join them.

### Join rows that have the same IDs

Convert dataframe to Pandas, as the groupby operation in `join_categorical_enum` isn't working properly with Modin:

In [ ]:
treat_df, pd = du.utils.convert_dataframe(treat_df, to='pandas')

In [ ]:
type(treat_df)

In [ ]:
treat_df = du.embedding.join_categorical_enum(treat_df, new_cat_embed_feat, inplace=True)
treat_df.head()

Reconvert dataframe to Modin:

In [ ]:
treat_df, pd = du.utils.convert_dataframe(treat_df, to='modin')

In [ ]:
type(treat_df)

In [ ]:
treat_df.dtypes

In [ ]:
treat_df.groupby(['patientunitstayid', 'ts']).count().nlargest(columns='treatmenttype', n=5).head()

In [ ]:
treat_df[treat_df.patientunitstayid == 1352520].head(10)

Comparing the output from the two previous cells with what we had before the `join_categorical_enum` method, we can see that all rows with duplicate IDs have been successfully joined.

### Clean column names

Standardize all column names to be on lower case, have spaces replaced by underscores and remove comas.

In [ ]:
treat_df.columns = du.data_processing.clean_naming(treat_df.columns)
treat_df.head()

### Save the dataframe

Save the dataframe before normalizing:

In [ ]:
treat_df.to_csv(f'{data_path}cleaned/unnormalized/diagnosis.csv')

Save the dataframe after normalizing:

In [ ]:
treat_df.to_csv(f'{data_path}cleaned/normalized/diagnosis.csv')

Confirm that everything is ok through the `describe` method:

In [ ]:
treat_df.describe().transpose()